In [1]:
from sqlite3 import connect
import sys
from os import curdir, path

In [2]:
modules_dir = path.join(curdir, '..', 'modules')
sys.path.append(modules_dir)

In [134]:
from db import read_pages
from heapq import nlargest

In [4]:
db = connect(path.join(curdir, '..', 'articles.sqlite'))

In [5]:
pages = db.execute("SELECT * FROM {}".format('articles')).fetchall()

In [6]:
import pandas as pd

In [7]:
# df = pd.DataFrame.from_records(pages, columns=['url', 'title', 'descr'])
# df.to_csv('data.csv')


In [8]:
titles = list(map(lambda x: x[1], pages))
descriptions = list(map(lambda x: x[2], pages))

In [9]:
print(titles[0])
print(descriptions[0])

12 советов, как заработать больше подписчиков в Instagram — Маркетинг на vc.ru
1. Продумайте контент-стратегию!


In [10]:
# from sklearn.model_selection import 

In [11]:
train_size = int(0.7 * len(titles))
test_size = len(titles) - train_size
data = list(zip(descriptions, titles))
train_data, test_data  = data[:train_size], data[train_size + 1:]

In [12]:
print(train_data[0], test_data[0])

('1. Продумайте контент-стратегию!', '12 советов, как заработать больше подписчиков в Instagram — Маркетинг на vc.ru') ('Сегодня расскажу несколько лайфхаков, которые можно использовать в CRM системе Bitrix24. Они помогут вам продавать больше, взять под контроль все каналы коммуникаций с клиентами, а также автоматизировать продажу с помощью триггеров и роботов.', 'Контроль рекламных каналов и автоматизация продаж — Маркетинг на vc.ru')


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csc_matrix

In [14]:
cv = CountVectorizer()

In [15]:
cv.fit(titles + descriptions)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
# data = list(zip(csc_matrix(map(lambda x: , descriptions), csc_matrix(cv.transform(titles))))

In [17]:
tokens = cv.get_feature_names()

In [18]:
word2num = {i:indx for indx, i in enumerate(tokens)} 

In [19]:
def tokenize_(s):
    a = []
    for s_ in s.split(' '):
        try:
            a.append(word2num[s_])
        except:
            continue
    return a

In [20]:
data = list(
    zip(
        map(lambda x: tokenize_(x), descriptions),
        map(lambda x: tokenize_(x), titles),
    )
)

In [21]:
data = list(filter(lambda x: len(x[0]) and len(x[1]), data))

In [22]:
train_size = int(0.7 * len(data))
test_size = len(data) - train_size
train_data, test_data  = data[:train_size], data[train_size + 1:]

In [23]:
train_data[0]

([325,
  594,
  1271,
  1107,
  587,
  980,
  577,
  611,
  587,
  662,
  445,
  146,
  364,
  634,
  1241,
  282,
  453,
  644,
  504,
  672],
 [1242, 291, 649, 587])

In [24]:
import numpy as np

In [25]:
eps = 1.0e-8

In [26]:
class Embedding:
    def __init__(self, matrix, epsilon=eps):
        self.data = matrix
        self.sigma = np.zeros(matrix.shape)
        self.sigma.fill(epsilon)
    @staticmethod
    def create(emb_size, vocab_size, rng = 0.001):
        rnd = np.random.randn(emb_size, vocab_size) * rng
        return Embedding(rnd)


In [27]:
emb = Embedding.create(256, len(tokens))

In [28]:
print(emb.sigma.shape)
print(emb.data.shape)
# print(emb.)

(256, 1283)
(256, 1283)


In [238]:
def top_k(w, k):
    res = agg(w)
    res = np.array(nlargest(k, enumerate(res), key=lambda x: x[1])[:k])[:, 0]
    return res

In [239]:
def backward_hinge(u, v, v_hat, gamma=1):
#     print(np.shape(u))
#     print(np.shape(v))
    loss = gamma - np.dot(u, v) + np.dot(u, v_hat)
    if loss > 0:
        return (v_hat - v, -u, u)

In [280]:
def update(embedding, idx, mask, delta, nu):
    p = emb.data[:, idx]
#     print(delta.shape)
#     print(mask.shape)    
    g = emb.sigma[:, idx]
    delta = np.dot(np.diag(delta), mask)
    emb.sigma[:, idx] += delta**2
    emb.data[:, idx] -= delta * nu / (np.sqrt(g) + eps)
# def update(emb, g_t, mask, lr=0.001, epsilon=1e-7):
#     w, h = emb.data[idx]
#     h_new = h + g_t * g_t
#     w_new = w - lr * g_t / (np.sqrt(h_new) + epsilon)
#     w_new = np.where(mask, w_new, w)
#     return w_new, h_new

In [281]:
def agg(w, mask=False):
#     print(np.shape(w))
    res = w.max(axis=1)
#     print('res', res.shape)
#     print(res)
    if mask:
        mask_res = w.max(axis=1, keepdims = 1) == w
#     print('mask', mask.shape)
        return res, mask_res # mask show values to update
    return res


In [282]:
def recall_at_k(emb, data, k = 10):
    n_test = len(data)
#     print(agg(emb.data[:, data[0][0]]))
#     print(np.shape(emb.data))
    descr_emb = [agg(emb.data[:, data[i][0]]) for i in range(n_test)]
    
    recall = []
    
    for i in range(n_test):
        title_emb = agg(emb.data[:, data[i][1]])
#         print('top_k', top_k(np.transpose(title_emb) * descr_emb, k))
#         print(i, top_k(np.transpose(title_emb) * descr_emb, k), i in top_k(np.transpose(title_emb) * descr_emb, k))
        recall.append(i in top_k(np.transpose(title_emb) * descr_emb, k))
#     print(recall)
    return np.count_nonzero(recall) / n_test

In [283]:
def train2(emb: Embedding, idx, nu):
#     print(idx[0])
#     print(idx)
#     print(np.shape(agg(emb.data[:, idx[0]])))
    embs, masks = [], []
    for ind in idx:
        res, mask = agg(emb.data[:, ind], mask=True)
        embs.append(res)
        masks.append(mask)
#     embs, masks = embs[:, 0], embs[:, 1]
#     print(np.shape(embs))
#     print(np.shape(embs))
    delta_s = backward_hinge(*embs)
#     print('delta_s', delta_s)
#     print(masks)
    if delta_s is None:
        return None
    
    for i, delta in zip(range(len(idx)), delta_s):
        update(emb, idx[i], masks[i], delta, nu)


In [284]:
def train1(emb, data, nu):
    first = np.random.permutation(len(data))
    second = np.roll(first, 1)
    for (f, s) in zip(first, second):
        u, v = data[f]
#         print(u, v)
#         print(u, v)
#         print(f, s)
        if np.random.randint(2):
            np.setdiff1d(v, u) 
        else: 
            np.setdiff1d(u, v)
        train2(emb, (u, v, data[s][1]), nu)

In [285]:
def train(emb, train_data, test_data, n_epochs, nu):
    for epoch in range(n_epochs):
        t = train1(emb, train_data, nu)
        recall = recall_at_k(emb, test_data)
        print("Epoch {} ({}): recall = {}\n".format(epoch, t, recall))

In [286]:
train(emb, train_data, test_data, 20, 1)

Epoch 0 (None): recall = 0.5588235294117647

Epoch 1 (None): recall = 0.5588235294117647

Epoch 2 (None): recall = 0.5588235294117647

Epoch 3 (None): recall = 0.5588235294117647

Epoch 4 (None): recall = 0.5588235294117647

Epoch 5 (None): recall = 0.5588235294117647

Epoch 6 (None): recall = 0.5588235294117647

Epoch 7 (None): recall = 0.5588235294117647

Epoch 8 (None): recall = 0.5588235294117647

Epoch 9 (None): recall = 0.5588235294117647

Epoch 10 (None): recall = 0.5588235294117647

Epoch 11 (None): recall = 0.5588235294117647

Epoch 12 (None): recall = 0.5588235294117647

Epoch 13 (None): recall = 0.5588235294117647

Epoch 14 (None): recall = 0.5588235294117647

Epoch 15 (None): recall = 0.5588235294117647

Epoch 16 (None): recall = 0.5588235294117647

Epoch 17 (None): recall = 0.5588235294117647

Epoch 18 (None): recall = 0.5588235294117647

Epoch 19 (None): recall = 0.5588235294117647

